# Exploratory Data Analysis (EDA) for Final Project

## Introduction

Exploratory Data Analysis (EDA) is a crucial step in the data analysis process. It involves summarizing the main characteristics of a dataset, often using visual methods. EDA helps in understanding the data better, uncovering patterns, spotting anomalies, testing hypotheses, and checking assumptions with the help of summary statistics and graphical representations. (Sahoo u. a., 2019)

In this notebook, we will explore a dataset containing information about pedestrian counts, weather conditions, working day indicators, events, etc. Our goal is to analyze relationships between these variables and derive meaningful insights.


# Data befor preprocessing

## Step 1: Importing Libraries and Loading the Dataset

In [92]:
import sqlite3
import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns

# from pandas_profiling import ProfileReport 
import ydata_profiling

In [93]:
# 1. Connect to the SQLite database
# Establish a connection to the SQLite database named 'data.db'.
connection = sqlite3.connect('data.db')
    
# 2. Load the data from the 'data' table
# Query the 'data' table to select only the 'location_id', 'pedestrians_count', and 'timestamp' columns.
# We exclude other columns because they either contain missing values or are not needed for our analysis.
wue_data = pd.read_sql('SELECT location_id, pedestrians_count, timestamp FROM data ORDER BY timestamp', connection)

# Extract the date from the timestamp and create a new 'date' column.
wue_data['date'] = wue_data['timestamp'].str.extract(r'(\d{4}-\d{2}-\d{2})')

# Split the timestamp to remove timezone information and create a new 'datetime' column.
wue_data['datetime'] = wue_data['timestamp'].str.split('+').str[0]
    
# 3. Load the weather data from the 'weather' table
# Query the 'weather' table to select the 'datetime', 'temp', 'humidity', and 'precip' columns.
weather_data = pd.read_sql('SELECT datetime, temp, humidity, precip FROM weather ORDER BY datetime', connection)
    
# 4. Load the event data from the 'events' table
# Query the 'events' table to select all columns and order the results by 'date'.
event_data = pd.read_sql('SELECT * FROM events ORDER BY date', connection)
    
# 5. Merge the data on the timestamp column
# Merge the 'wue_data' and 'event_data' DataFrames on the 'date' column using a left join.
data = pd.merge(wue_data, event_data, on='date', how='left')

# Merge the resulting DataFrame with 'weather_data' on the 'datetime' column using a left join.
data = pd.merge(data, weather_data, on='datetime', how='left')

# Sort the data by 'timestamp' and 'location_id' to organize the DataFrame.
data.sort_values(by=['timestamp', 'location_id'], inplace=True)

# 5a. Save the data as a CSV file
# Save the merged DataFrame to a CSV file named 'merged_data.csv' without the index column.
data.to_csv('merged_data.csv', index=False) 
    
# 6. Close the database connection
# Close the connection to the SQLite database.
connection.close()

data from load_data.py

In [94]:
# converted data to a pandas DataFrame
df = pd.read_csv("merged_data.csv")

## Step 2: Understanding the Data Structure

We need to understand our dataset. 

In [4]:
# Display the shape of DataFrame
df.shape

(13919, 13)

In [69]:
#This command allows us to inspect all the columns
pd.set_option('display.max_columns', None)

In [5]:
# Display the first few rows of the dataset
df.head()

,location_id,pedestrians_count,timestamp,date,datetime,holiday,workday,green_market,specialities_market,event,temp,humidity,precip
0,135.0,1346,2024-01-01T00:00:00+01:00,2024-01-01,2024-01-01T00:00:00,1,1,0,0,0,4.8,87.50,0.0
1,470.0,388,2024-01-01T00:00:00+01:00,2024-01-01,2024-01-01T00:00:00,1,1,0,0,0,4.8,87.50,0.0
2,476.0,1197,2024-01-01T00:00:00+01:00,2024-01-01,2024-01-01T00:00:00,1,1,0,0,0,4.8,87.50,0.0
3,135.0,772,2024-01-01T01:00:00+01:00,2024-01-01,2024-01-01T01:00:00,1,1,0,0,0,4.9,80.73,0.0
4,470.0,161,2024-01-01T01:00:00+01:00,2024-01-01,2024-01-01T01:00:00,1,1,0,0,0,4.9,80.73,0.0


In [6]:
# display the last 5 observations of the dataset
df.tail()

,location_id,pedestrians_count,timestamp,date,datetime,holiday,workday,green_market,specialities_market,event,temp,humidity,precip
13914,470.0,1712,2024-07-13T15:00:00+02:00,2024-07-13,2024-07-13T15:00:00,0,0,1,1,1,21.0,49.39,0.0
13915,476.0,2574,2024-07-13T15:00:00+02:00,2024-07-13,2024-07-13T15:00:00,0,0,1,1,1,21.0,49.39,0.0
13916,135.0,4831,2024-07-13T16:00:00+02:00,2024-07-13,2024-07-13T16:00:00,0,0,1,1,1,22.0,46.45,0.0
13917,470.0,1496,2024-07-13T16:00:00+02:00,2024-07-13,2024-07-13T16:00:00,0,0,1,1,1,22.0,46.45,0.0
13918,476.0,2601,2024-07-13T16:00:00+02:00,2024-07-13,2024-07-13T16:00:00,0,0,1,1,1,22.0,46.45,0.0


In [7]:
# Displaying the structure of the dataset to understand the data type and information about data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13919 entries, 0 to 13918
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   location_id          13919 non-null  float64
 1   pedestrians_count    13919 non-null  int64  
 2   timestamp            13919 non-null  object 
 3   date                 13919 non-null  object 
 4   datetime             13919 non-null  object 
 5   holiday              13919 non-null  int64  
 6   workday              13919 non-null  int64  
 7   green_market         13919 non-null  int64  
 8   specialities_market  13919 non-null  int64  
 9   event                13919 non-null  int64  
 10  temp                 13919 non-null  float64
 11  humidity             13919 non-null  float64
 12  precip               13919 non-null  float64
dtypes: float64(4), int64(6), object(3)
memory usage: 1.4+ MB


In [8]:
# Check for Duplication
df.nunique()

location_id               3
pedestrians_count      2915
timestamp              4645
date                    195
datetime               4645
holiday                   2
workday                   2
green_market              2
specialities_market       2
event                     2
temp                    377
humidity               3070
precip                  404
dtype: int64

In [9]:
# Missing Values Calculation
df.isnull().sum()

location_id            0
pedestrians_count      0
timestamp              0
date                   0
datetime               0
holiday                0
workday                0
green_market           0
specialities_market    0
event                  0
temp                   0
humidity               0
precip                 0
dtype: int64

In [10]:
# to calculate the percentage of missing values in each column
(df.isnull().sum()/(len(data)))*100

location_id            0.0
pedestrians_count      0.0
timestamp              0.0
date                   0.0
datetime               0.0
holiday                0.0
workday                0.0
green_market           0.0
specialities_market    0.0
event                  0.0
temp                   0.0
humidity               0.0
precip                 0.0
dtype: float64

In [71]:
# Summary statistics for numerical features
df.describe()

,pedestrian_count,temp,humidity
count,5.000000,5.000000,5.000000
mean,122.000000,24.800000,46.400000
std,19.235384,3.962323,6.107373
min,100.000000,20.000000,40.000000
25%,110.000000,22.000000,42.000000
50%,120.000000,25.000000,45.000000
75%,130.000000,27.000000,50.000000
max,150.000000,30.000000,55.000000


# Data after Preprocessing

We take the CSV file that was saved during the pipeline. the data is tep after preproccessing. The location_id, date, temperature and humidity are saved as non-scaled values so that we can use them later in the visualization step

## Step 1: Loading the Dataset

In [72]:
# Loading the dataset
data_pre = pd.read_csv("data\processed\eda_data.csv")

<>:2: SyntaxWarning:

invalid escape sequence '\p'

<>:2: SyntaxWarning:

invalid escape sequence '\p'

C:\Users\gavar\AppData\Local\Temp\ipykernel_6584\3440759093.py:2: SyntaxWarning:

invalid escape sequence '\p'



## Step 2: Understanding the Data Structure

In [73]:
# Display the first few rows of the dataset
data_pre.head()

,holiday,workday,green_market,specialities_market,event,precip,pedestrians_count_lag_1,pedestrians_count_lag_2,pedestrians_count_lag_3,pedestrians_count_lag_4,pedestrians_count_lag_5,event_lag_1,holiday_lag_1,workday_lag_1,event_lag_2,holiday_lag_2,workday_lag_2,event_lag_3,holiday_lag_3,workday_lag_3,event_lag_4,holiday_lag_4,workday_lag_4,event_lag_5,holiday_lag_5,workday_lag_5,event_lead_1,holiday_lead_1,workday_lead_1,event_lead_2,holiday_lead_2,workday_lead_2,event_lead_3,holiday_lead_3,workday_lead_3,event_lead_4,holiday_lead_4,workday_lead_4,event_lead_5,holiday_lead_5,workday_lead_5,temp_lag_1,humidity_lag_1,precip_lag_1,temp_lag_2,humidity_lag_2,precip_lag_2,temp_lag_3,humidity_lag_3,precip_lag_3,temp_lag_4,humidity_lag_4,precip_lag_4,temp_lag_5,humidity_lag_5,precip_lag_5,weekday_Friday,weekday_Monday,weekday_Saturday,weekday_Sunday,weekday_Thursday,weekday_Tuesday,weekday_Wednesday,year,month,day,hour,location_id,temp,humidity,series
0,3.462246,0.628236,-1.147189,-1.147189,-0.430474,-0.19875,-0.667769,0.001323,0.466675,-0.419384,0.629719,-0.430327,3.460057,0.628236,-0.43018,3.457872,0.628236,-0.430033,3.45569,0.628236,-0.429886,3.453512,0.628236,-0.42974,3.451338,0.628236,-0.43062,3.464439,0.628236,-0.430767,3.466636,0.628236,-0.430914,3.468836,0.628236,-0.431061,3.47104,0.628236,-0.431207,3.473248,0.628236,-0.613190,0.309481,-0.19875,-0.613108,0.309548,-0.19875,-0.628602,0.764533,-0.19875,-0.628517,0.764510,-0.19875,-0.628433,0.764487,-0.19875,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2024,1,1,1,476.0,4.9,80.73,697
1,3.462246,0.628236,-1.147189,-1.147189,-0.430474,-0.19875,-0.080760,-0.667835,0.001215,0.466651,-0.419514,-0.430327,3.460057,0.628236,-0.43018,3.457872,0.628236,-0.430033,3.45569,0.628236,-0.429886,3.453512,0.628236,-0.42974,3.451338,0.628236,-0.43062,3.464439,0.628236,-0.430767,3.466636,0.628236,-0.430914,3.468836,0.628236,-0.431061,3.47104,0.628236,-0.431207,3.473248,0.628236,-0.613190,0.309481,-0.19875,-0.613108,0.309548,-0.19875,-0.613024,0.309567,-0.19875,-0.628517,0.764510,-0.19875,-0.628433,0.764487,-0.19875,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2024,1,1,2,135.0,6.2,75.01,211
2,3.462246,0.628236,-1.147189,-1.147189,-0.430474,-0.19875,-0.613010,-0.080816,-0.667953,0.001181,0.466529,-0.430327,3.460057,0.628236,-0.43018,3.457872,0.628236,-0.430033,3.45569,0.628236,-0.429886,3.453512,0.628236,-0.42974,3.451338,0.628236,-0.43062,3.464439,0.628236,-0.430767,3.466636,0.628236,-0.430914,3.468836,0.628236,-0.431061,3.47104,0.628236,-0.431207,3.473248,0.628236,-0.410667,-0.074903,-0.19875,-0.613108,0.309548,-0.19875,-0.613024,0.309567,-0.19875,-0.612939,0.309549,-0.19875,-0.628433,0.764487,-0.19875,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2024,1,1,2,470.0,6.2,75.01,118
3,3.462246,0.628236,-1.147189,-1.147189,-0.430474,-0.19875,-0.714861,-0.613075,-0.080925,-0.668000,0.001055,-0.430327,3.460057,0.628236,-0.43018,3.457872,0.628236,-0.430033,3.45569,0.628236,-0.429886,3.453512,0.628236,-0.42974,3.451338,0.628236,-0.43062,3.464439,0.628236,-0.430767,3.466636,0.628236,-0.430914,3.468836,0.628236,-0.431061,3.47104,0.628236,-0.431207,3.473248,0.628236,-0.410667,-0.074903,-0.19875,-0.410588,-0.074849,-0.19875,-0.613024,0.309567,-0.19875,-0.612939,0.309549,-0.19875,-0.612855,0.309531,-0.19875,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2024,1,1,2,476.0,6.2,75.01,243
4,3.462246,0.628236,-1.147189,-1.147189,-0.430474,-0.19875,-0.577965,-0.714928,-0.613192,-0.080960,-0.668132,-0.430327,3.460057,0.628236,-0.43018,3.457872,0.628236,-0.430033,3.45569,0.628236,-0.429886,3.453512,0.628236,-0.42974,3.451338,0.628236,-0.43062,3.464439,0.628236,-0.430767,3.466636,0.628236,-0.430914,3.468836,0.628236,-0.431061,3.47104,0.628236,-0.431207,3.473248,0.628236,-0.410667,-0.074903,-0.19875,-0.410588,-0.074849,-0.19875,-0.410507,-0.074835,-0.19875,-0.612939,0.309549,-0.19875,-0.612855,0.309531,-0.19875,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2024,1,1,3,135.0,6.2,75.55,173


In [87]:
# Displaying the structure of the dataset
data_pre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13897 entries, 0 to 13896
Data columns (total 76 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   holiday                  13897 non-null  float64       
 1   workday                  13897 non-null  float64       
 2   green_market             13897 non-null  float64       
 3   specialities_market      13897 non-null  float64       
 4   event                    13897 non-null  float64       
 5   precip                   13897 non-null  float64       
 6   pedestrians_count_lag_1  13897 non-null  float64       
 7   pedestrians_count_lag_2  13897 non-null  float64       
 8   pedestrians_count_lag_3  13897 non-null  float64       
 9   pedestrians_count_lag_4  13897 non-null  float64       
 10  pedestrians_count_lag_5  13897 non-null  float64       
 11  event_lag_1              13897 non-null  float64       
 12  holiday_lag_1            13897 n

After Preprocessing are all of our data complete, without missing values

In [88]:
# Checking for missing values
data_pre.isnull().sum()

holiday                    0
workday                    0
green_market               0
specialities_market        0
event                      0
precip                     0
pedestrians_count_lag_1    0
pedestrians_count_lag_2    0
pedestrians_count_lag_3    0
pedestrians_count_lag_4    0
pedestrians_count_lag_5    0
event_lag_1                0
holiday_lag_1              0
workday_lag_1              0
event_lag_2                0
holiday_lag_2              0
workday_lag_2              0
event_lag_3                0
holiday_lag_3              0
workday_lag_3              0
event_lag_4                0
holiday_lag_4              0
workday_lag_4              0
event_lag_5                0
holiday_lag_5              0
workday_lag_5              0
event_lead_1               0
holiday_lead_1             0
workday_lead_1             0
event_lead_2               0
holiday_lead_2             0
workday_lead_2             0
event_lead_3               0
holiday_lead_3             0
workday_lead_3

## Step 3: Summary Statistics

In [15]:
#This command allows us to inspect all the rows
pd.set_option('display.max_rows', None)

In [74]:
# Summary statistics for numerical features
data_pre.describe().T

,count,mean,std,min,25%,50%,75%,max
holiday,13897.0,-0.057778,0.901868,-0.288830,-0.288830,-0.288830,-0.288830,3.462246
workday,13897.0,0.006504,0.996877,-1.591758,-1.591758,0.628236,0.628236,0.628236
green_market,13897.0,0.008909,0.998768,-1.147189,-1.147189,0.871696,0.871696,0.871696
specialities_market,13897.0,0.008909,0.998768,-1.147189,-1.147189,0.871696,0.871696,0.871696
event,13897.0,0.111825,1.095087,-0.430474,-0.430474,-0.430474,-0.430474,2.323022
precip,13897.0,0.010804,1.173618,-0.198750,-0.198750,-0.198750,-0.198750,35.186123
pedestrians_count_lag_1,13897.0,0.015209,0.994227,-0.844090,-0.783856,-0.363312,0.548961,6.000698
pedestrians_count_lag_2,13897.0,0.015114,0.994232,-0.844159,-0.783924,-0.363373,0.548915,6.000743
pedestrians_count_lag_3,13897.0,0.015019,0.994252,-0.844280,-0.784044,-0.363487,0.548815,6.000722
pedestrians_count_lag_4,13897.0,0.014870,0.994229,-0.844331,-0.784093,-0.364623,0.548793,6.000811


## Step 4: Modify data for the following analysis 

In [76]:
# Convert year, month, day, and hour columns to a single datetime column
data_pre['datetime'] = pd.to_datetime(data_pre[['year', 'month', 'day', 'hour']])
# Convert year, month, and day columns to a date column
data_pre['date'] = pd.to_datetime(data_pre[['year', 'month', 'day']])
# Display the first 5 rows of the modified DataFrame
data_pre.head(5)

,holiday,workday,green_market,specialities_market,event,precip,pedestrians_count_lag_1,pedestrians_count_lag_2,pedestrians_count_lag_3,pedestrians_count_lag_4,pedestrians_count_lag_5,event_lag_1,holiday_lag_1,workday_lag_1,event_lag_2,holiday_lag_2,workday_lag_2,event_lag_3,holiday_lag_3,workday_lag_3,event_lag_4,holiday_lag_4,workday_lag_4,event_lag_5,holiday_lag_5,workday_lag_5,event_lead_1,holiday_lead_1,workday_lead_1,event_lead_2,holiday_lead_2,workday_lead_2,event_lead_3,holiday_lead_3,workday_lead_3,event_lead_4,holiday_lead_4,workday_lead_4,event_lead_5,holiday_lead_5,workday_lead_5,temp_lag_1,humidity_lag_1,precip_lag_1,temp_lag_2,humidity_lag_2,precip_lag_2,temp_lag_3,humidity_lag_3,precip_lag_3,temp_lag_4,humidity_lag_4,precip_lag_4,temp_lag_5,humidity_lag_5,precip_lag_5,weekday_Friday,weekday_Monday,weekday_Saturday,weekday_Sunday,weekday_Thursday,weekday_Tuesday,weekday_Wednesday,year,month,day,hour,location_id,temp,humidity,series,datetime,date
0,3.462246,0.628236,-1.147189,-1.147189,-0.430474,-0.19875,-0.667769,0.001323,0.466675,-0.419384,0.629719,-0.430327,3.460057,0.628236,-0.43018,3.457872,0.628236,-0.430033,3.45569,0.628236,-0.429886,3.453512,0.628236,-0.42974,3.451338,0.628236,-0.43062,3.464439,0.628236,-0.430767,3.466636,0.628236,-0.430914,3.468836,0.628236,-0.431061,3.47104,0.628236,-0.431207,3.473248,0.628236,-0.613190,0.309481,-0.19875,-0.613108,0.309548,-0.19875,-0.628602,0.764533,-0.19875,-0.628517,0.764510,-0.19875,-0.628433,0.764487,-0.19875,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2024,1,1,1,476.0,4.9,80.73,697,2024-01-01 01:00:00,2024-01-01
1,3.462246,0.628236,-1.147189,-1.147189,-0.430474,-0.19875,-0.080760,-0.667835,0.001215,0.466651,-0.419514,-0.430327,3.460057,0.628236,-0.43018,3.457872,0.628236,-0.430033,3.45569,0.628236,-0.429886,3.453512,0.628236,-0.42974,3.451338,0.628236,-0.43062,3.464439,0.628236,-0.430767,3.466636,0.628236,-0.430914,3.468836,0.628236,-0.431061,3.47104,0.628236,-0.431207,3.473248,0.628236,-0.613190,0.309481,-0.19875,-0.613108,0.309548,-0.19875,-0.613024,0.309567,-0.19875,-0.628517,0.764510,-0.19875,-0.628433,0.764487,-0.19875,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2024,1,1,2,135.0,6.2,75.01,211,2024-01-01 02:00:00,2024-01-01
2,3.462246,0.628236,-1.147189,-1.147189,-0.430474,-0.19875,-0.613010,-0.080816,-0.667953,0.001181,0.466529,-0.430327,3.460057,0.628236,-0.43018,3.457872,0.628236,-0.430033,3.45569,0.628236,-0.429886,3.453512,0.628236,-0.42974,3.451338,0.628236,-0.43062,3.464439,0.628236,-0.430767,3.466636,0.628236,-0.430914,3.468836,0.628236,-0.431061,3.47104,0.628236,-0.431207,3.473248,0.628236,-0.410667,-0.074903,-0.19875,-0.613108,0.309548,-0.19875,-0.613024,0.309567,-0.19875,-0.612939,0.309549,-0.19875,-0.628433,0.764487,-0.19875,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2024,1,1,2,470.0,6.2,75.01,118,2024-01-01 02:00:00,2024-01-01
3,3.462246,0.628236,-1.147189,-1.147189,-0.430474,-0.19875,-0.714861,-0.613075,-0.080925,-0.668000,0.001055,-0.430327,3.460057,0.628236,-0.43018,3.457872,0.628236,-0.430033,3.45569,0.628236,-0.429886,3.453512,0.628236,-0.42974,3.451338,0.628236,-0.43062,3.464439,0.628236,-0.430767,3.466636,0.628236,-0.430914,3.468836,0.628236,-0.431061,3.47104,0.628236,-0.431207,3.473248,0.628236,-0.410667,-0.074903,-0.19875,-0.410588,-0.074849,-0.19875,-0.613024,0.309567,-0.19875,-0.612939,0.309549,-0.19875,-0.612855,0.309531,-0.19875,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2024,1,1,2,476.0,6.2,75.01,243,2024-01-01 02:00:00,2024-01-01
4,3.462246,0.628236,-1.147189,-1.147189,-0.430474,-0.19875,-0.577965,-0.714928,-0.613192,-0.080960,-0.668132,-0.430327,3.460057,0.628236,-0.43018,3.457872,0.628236,-0.430033,3.45569,0.628236,-0.429886,3.453512,0.628236,-0.42974,3.451338,0.628236,-0.43062,3.464439,0.628236,-0.430767,3.466636,0.628236,-0.430914,3.468836,0.628236,-0.431061,3.47104,0.628236,-0.431207,3.473248,0.628236,-0.410667,-0.074903,-0.19875,-0.410588,-0.074849,-0.19875,-0.410507,-0.074835,-0.19875,-0.612939,0.309549,-0.19875,-0.612855,0.309531,-0.19875,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2024,1,1,3,135.0,6.2,

From database we get the street names for the corresponding location_id or further analysis

In [22]:
# Establish a connection to the SQLite database named 'data.db'
connection = sqlite3.connect('data.db')
# 1. Load the event data from the 'events' table
# Query the 'data' table to select 'location_name' and 'location_id'
location = pd.read_sql('SELECT location_name, location_id FROM data', connection)

# 2. Close the database connection
connection.close()

In [77]:
# Display the first 5 rows of the DataFrame
location.head(5)

,location_name,location_id
0,Schönbornstraße,135.0
1,Spiegelstraße,470.0
2,Kaiserstraße,476.0
3,Schönbornstraße,135.0
4,Spiegelstraße,470.0


In [78]:
# Mapping for the location_id to location_name
mapping = {
    135: 'Schönbornstraße',
    470: 'Spiegelstraße',
    476: 'Kaiserstraße'
}

# Add new column location_name based on location_id
data_pre['location_name'] = data_pre['location_id'].map(mapping)

In [79]:
# Display the first 5 rows of the modified DataFrame
data_pre.head(5)

,holiday,workday,green_market,specialities_market,event,precip,pedestrians_count_lag_1,pedestrians_count_lag_2,pedestrians_count_lag_3,pedestrians_count_lag_4,pedestrians_count_lag_5,event_lag_1,holiday_lag_1,workday_lag_1,event_lag_2,holiday_lag_2,workday_lag_2,event_lag_3,holiday_lag_3,workday_lag_3,event_lag_4,holiday_lag_4,workday_lag_4,event_lag_5,holiday_lag_5,workday_lag_5,event_lead_1,holiday_lead_1,workday_lead_1,event_lead_2,holiday_lead_2,workday_lead_2,event_lead_3,holiday_lead_3,workday_lead_3,event_lead_4,holiday_lead_4,workday_lead_4,event_lead_5,holiday_lead_5,workday_lead_5,temp_lag_1,humidity_lag_1,precip_lag_1,temp_lag_2,humidity_lag_2,precip_lag_2,temp_lag_3,humidity_lag_3,precip_lag_3,temp_lag_4,humidity_lag_4,precip_lag_4,temp_lag_5,humidity_lag_5,precip_lag_5,weekday_Friday,weekday_Monday,weekday_Saturday,weekday_Sunday,weekday_Thursday,weekday_Tuesday,weekday_Wednesday,year,month,day,hour,location_id,temp,humidity,series,datetime,date,location_name
0,3.462246,0.628236,-1.147189,-1.147189,-0.430474,-0.19875,-0.667769,0.001323,0.466675,-0.419384,0.629719,-0.430327,3.460057,0.628236,-0.43018,3.457872,0.628236,-0.430033,3.45569,0.628236,-0.429886,3.453512,0.628236,-0.42974,3.451338,0.628236,-0.43062,3.464439,0.628236,-0.430767,3.466636,0.628236,-0.430914,3.468836,0.628236,-0.431061,3.47104,0.628236,-0.431207,3.473248,0.628236,-0.613190,0.309481,-0.19875,-0.613108,0.309548,-0.19875,-0.628602,0.764533,-0.19875,-0.628517,0.764510,-0.19875,-0.628433,0.764487,-0.19875,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2024,1,1,1,476.0,4.9,80.73,697,2024-01-01 01:00:00,2024-01-01,Kaiserstraße
1,3.462246,0.628236,-1.147189,-1.147189,-0.430474,-0.19875,-0.080760,-0.667835,0.001215,0.466651,-0.419514,-0.430327,3.460057,0.628236,-0.43018,3.457872,0.628236,-0.430033,3.45569,0.628236,-0.429886,3.453512,0.628236,-0.42974,3.451338,0.628236,-0.43062,3.464439,0.628236,-0.430767,3.466636,0.628236,-0.430914,3.468836,0.628236,-0.431061,3.47104,0.628236,-0.431207,3.473248,0.628236,-0.613190,0.309481,-0.19875,-0.613108,0.309548,-0.19875,-0.613024,0.309567,-0.19875,-0.628517,0.764510,-0.19875,-0.628433,0.764487,-0.19875,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2024,1,1,2,135.0,6.2,75.01,211,2024-01-01 02:00:00,2024-01-01,Schönbornstraße
2,3.462246,0.628236,-1.147189,-1.147189,-0.430474,-0.19875,-0.613010,-0.080816,-0.667953,0.001181,0.466529,-0.430327,3.460057,0.628236,-0.43018,3.457872,0.628236,-0.430033,3.45569,0.628236,-0.429886,3.453512,0.628236,-0.42974,3.451338,0.628236,-0.43062,3.464439,0.628236,-0.430767,3.466636,0.628236,-0.430914,3.468836,0.628236,-0.431061,3.47104,0.628236,-0.431207,3.473248,0.628236,-0.410667,-0.074903,-0.19875,-0.613108,0.309548,-0.19875,-0.613024,0.309567,-0.19875,-0.612939,0.309549,-0.19875,-0.628433,0.764487,-0.19875,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2024,1,1,2,470.0,6.2,75.01,118,2024-01-01 02:00:00,2024-01-01,Spiegelstraße
3,3.462246,0.628236,-1.147189,-1.147189,-0.430474,-0.19875,-0.714861,-0.613075,-0.080925,-0.668000,0.001055,-0.430327,3.460057,0.628236,-0.43018,3.457872,0.628236,-0.430033,3.45569,0.628236,-0.429886,3.453512,0.628236,-0.42974,3.451338,0.628236,-0.43062,3.464439,0.628236,-0.430767,3.466636,0.628236,-0.430914,3.468836,0.628236,-0.431061,3.47104,0.628236,-0.431207,3.473248,0.628236,-0.410667,-0.074903,-0.19875,-0.410588,-0.074849,-0.19875,-0.613024,0.309567,-0.19875,-0.612939,0.309549,-0.19875,-0.612855,0.309531,-0.19875,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2024,1,1,2,476.0,6.2,75.01,243,2024-01-01 02:00:00,2024-01-01,Kaiserstraße
4,3.462246,0.628236,-1.147189,-1.147189,-0.430474,-0.19875,-0.577965,-0.714928,-0.613192,-0.080960,-0.668132,-0.430327,3.460057,0.628236,-0.43018,3.457872,0.628236,-0.430033,3.45569,0.628236,-0.429886,3.453512,0.628236,-0.42974,3.451338,0.628236,-0.43062,3.464439,0.628236,-0.430767,3.466636,0.628236,-0.430914,3.468836,0.628236,-0.431061,3.47104,0.628236,-0.431207,3.473248,0.628236,-0.410667,-0.074903,-0.19875,-0.410588,-0.074849,-0.19875,-0.410507,-0.074835,-0.19875,-0.612939,0.309549,-0.19875,-0.6128

In [80]:
# Calculate the day of the week (Monday=0, Sunday=6) from the 'date' column
data_pre['day_of_week'] = data_pre['date'].dt.dayofweek

# Add day names corresponding to the day of the week
days = {0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday', 5: 'Saturday', 6: 'Sunday'}
data_pre['day_name'] = data_pre['day_of_week'].map(days)
# Display the first 5 rows of the modified DataFrame
data_pre.head(5)

,holiday,workday,green_market,specialities_market,event,precip,pedestrians_count_lag_1,pedestrians_count_lag_2,pedestrians_count_lag_3,pedestrians_count_lag_4,pedestrians_count_lag_5,event_lag_1,holiday_lag_1,workday_lag_1,event_lag_2,holiday_lag_2,workday_lag_2,event_lag_3,holiday_lag_3,workday_lag_3,event_lag_4,holiday_lag_4,workday_lag_4,event_lag_5,holiday_lag_5,workday_lag_5,event_lead_1,holiday_lead_1,workday_lead_1,event_lead_2,holiday_lead_2,workday_lead_2,event_lead_3,holiday_lead_3,workday_lead_3,event_lead_4,holiday_lead_4,workday_lead_4,event_lead_5,holiday_lead_5,workday_lead_5,temp_lag_1,humidity_lag_1,precip_lag_1,temp_lag_2,humidity_lag_2,precip_lag_2,temp_lag_3,humidity_lag_3,precip_lag_3,temp_lag_4,humidity_lag_4,precip_lag_4,temp_lag_5,humidity_lag_5,precip_lag_5,weekday_Friday,weekday_Monday,weekday_Saturday,weekday_Sunday,weekday_Thursday,weekday_Tuesday,weekday_Wednesday,year,month,day,hour,location_id,temp,humidity,series,datetime,date,location_name,day_of_week,day_name
0,3.462246,0.628236,-1.147189,-1.147189,-0.430474,-0.19875,-0.667769,0.001323,0.466675,-0.419384,0.629719,-0.430327,3.460057,0.628236,-0.43018,3.457872,0.628236,-0.430033,3.45569,0.628236,-0.429886,3.453512,0.628236,-0.42974,3.451338,0.628236,-0.43062,3.464439,0.628236,-0.430767,3.466636,0.628236,-0.430914,3.468836,0.628236,-0.431061,3.47104,0.628236,-0.431207,3.473248,0.628236,-0.613190,0.309481,-0.19875,-0.613108,0.309548,-0.19875,-0.628602,0.764533,-0.19875,-0.628517,0.764510,-0.19875,-0.628433,0.764487,-0.19875,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2024,1,1,1,476.0,4.9,80.73,697,2024-01-01 01:00:00,2024-01-01,Kaiserstraße,0,Monday
1,3.462246,0.628236,-1.147189,-1.147189,-0.430474,-0.19875,-0.080760,-0.667835,0.001215,0.466651,-0.419514,-0.430327,3.460057,0.628236,-0.43018,3.457872,0.628236,-0.430033,3.45569,0.628236,-0.429886,3.453512,0.628236,-0.42974,3.451338,0.628236,-0.43062,3.464439,0.628236,-0.430767,3.466636,0.628236,-0.430914,3.468836,0.628236,-0.431061,3.47104,0.628236,-0.431207,3.473248,0.628236,-0.613190,0.309481,-0.19875,-0.613108,0.309548,-0.19875,-0.613024,0.309567,-0.19875,-0.628517,0.764510,-0.19875,-0.628433,0.764487,-0.19875,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2024,1,1,2,135.0,6.2,75.01,211,2024-01-01 02:00:00,2024-01-01,Schönbornstraße,0,Monday
2,3.462246,0.628236,-1.147189,-1.147189,-0.430474,-0.19875,-0.613010,-0.080816,-0.667953,0.001181,0.466529,-0.430327,3.460057,0.628236,-0.43018,3.457872,0.628236,-0.430033,3.45569,0.628236,-0.429886,3.453512,0.628236,-0.42974,3.451338,0.628236,-0.43062,3.464439,0.628236,-0.430767,3.466636,0.628236,-0.430914,3.468836,0.628236,-0.431061,3.47104,0.628236,-0.431207,3.473248,0.628236,-0.410667,-0.074903,-0.19875,-0.613108,0.309548,-0.19875,-0.613024,0.309567,-0.19875,-0.612939,0.309549,-0.19875,-0.628433,0.764487,-0.19875,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2024,1,1,2,470.0,6.2,75.01,118,2024-01-01 02:00:00,2024-01-01,Spiegelstraße,0,Monday
3,3.462246,0.628236,-1.147189,-1.147189,-0.430474,-0.19875,-0.714861,-0.613075,-0.080925,-0.668000,0.001055,-0.430327,3.460057,0.628236,-0.43018,3.457872,0.628236,-0.430033,3.45569,0.628236,-0.429886,3.453512,0.628236,-0.42974,3.451338,0.628236,-0.43062,3.464439,0.628236,-0.430767,3.466636,0.628236,-0.430914,3.468836,0.628236,-0.431061,3.47104,0.628236,-0.431207,3.473248,0.628236,-0.410667,-0.074903,-0.19875,-0.410588,-0.074849,-0.19875,-0.613024,0.309567,-0.19875,-0.612939,0.309549,-0.19875,-0.612855,0.309531,-0.19875,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2024,1,1,2,476.0,6.2,75.01,243,2024-01-01 02:00:00,2024-01-01,Kaiserstraße,0,Monday
4,3.462246,0.628236,-1.147189,-1.147189,-0.430474,-0.19875,-0.577965,-0.714928,-0.613192,-0.080960,-0.668132,-0.430327,3.460057,0.628236,-0.43018,3.457872,0.628236,-0.430033,3.45569,0.628236,-0.429886,3.453512,0.628236,-0.42974,3.451338,0.628236,-0.43062,3.464439,0.628236,-0.430767,3.466636,0.628236,-0.430914,3.468836,0.628236,-0.431061,3.47104,0.628236,-0.431207,3.473248,0.628236,-0.410667,-0.074903,-0.19875,-0.410588,-0.074849,-0.19875,-0.4105

In [81]:
# Calculate average pedestrian count for each day of the week
avg_pedestrian_count = data_pre.groupby('day_name')['series'].mean().reset_index()

# Sort the weekdays in the correct order
avg_pedestrian_count['day_name'] = pd.Categorical(avg_pedestrian_count['day_name'], 
                                                  categories=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'],
                                                  ordered=True)
avg_pedestrian_count = avg_pedestrian_count.sort_values('day_name')

## Step 5: Univariate Analysis

We will analyze individual variables to understand their distributions and characteristics.

### Pedestrian Count Distribution

Diagram indicates a negative offset (left offset)

In [83]:
fig = px.histogram(data_pre, x='series', nbins=30, title='Distribution of Pedestrian Counts',
                   labels={'series': 'Pedestrian Count', 'count': 'Frequency'})
fig.update_layout(xaxis_title='Pedestrian Count', yaxis_title='Frequency', width=800, height=400)
fig.show()

### Weather Information

In [85]:
# Visualizing weather information: temperature and humidity
fig = go.Figure()

fig.add_trace(go.Histogram(x=data_pre['temp'], name='Temperature', marker_color='blue', nbinsx=20))
fig.add_trace(go.Histogram(x=data_pre['humidity'], name='Humidity', marker_color='green', nbinsx=20))

# Update layout
fig.update_layout(title='Distribution of Temperature and Humidity',
                  xaxis_title='Value',
                  yaxis_title='Frequency',
                  bargap=0.2,  # gap between bars of adjacent location coordinates
                  bargroupgap=0.1)  # gap between bars of the same location coordinates

fig.show()

## Step 6: Bivariate Analysis

### Impact of Day of the Week on Pedestrian Count

In [65]:
# Create a bar chart and add a trend line
fig = go.Figure()

# Add bar chart
fig.add_trace(go.Bar(
    x=avg_pedestrian_count['day_name'], 
    y=avg_pedestrian_count['series'], 
    name='Average Pedestrian Count'
))

# Add trend line
fig.add_trace(go.Scatter(
    x=avg_pedestrian_count['day_name'], 
    y=avg_pedestrian_count['series'], 
    mode='lines+markers',
    name='Trend',
    line=dict(color='red')
))

# Customize layout
fig.update_layout(
    title='Average Pedestrian Count by Day of the Week with Trend Line',
    xaxis_title='Day of the Week',
    yaxis_title='Average Pedestrian Count',
    showlegend=True
)

fig.show()

### Pedestrian count by location

In [27]:
# Create subplots with shared X-axis and subplot titles
fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.05,
                    subplot_titles=("Kaiserstraße", "Schönbornstraße", "Spiegelstraße"))

# Iterate through unique street names in data_pre and add traces to subplots
for i, street in enumerate(data_pre['location_name'].unique()):
    df_street = data_pre[data_pre['location_name'] == street]
    fig.add_trace(
        go.Scatter(x=df_street['datetime'], y=df_street['series'], name=street),
        row=i+1, col=1
    )

# Customize layout
fig.update_layout(height=900, title_text="Pedestrian count by location")
fig.update_yaxes(title_text="Pedestrian Count") 

We aggregate the pedestrian count per day in order to visualize the relationship between pedestrian count and date.

In [28]:
aggregated_df = data_pre.groupby(['location_name', 'date']).agg({'series': 'sum'}).reset_index()
aggregated_df.head(5)

,location_name,date,series
0,Kaiserstraße,2024-01-01,7190
1,Kaiserstraße,2024-01-02,16597
2,Kaiserstraße,2024-01-03,19225
3,Kaiserstraße,2024-01-04,21849
4,Kaiserstraße,2024-01-05,24936


In [32]:
fig = px.line(aggregated_df, x="date", y="series",color='location_name', title='Pedestrian count by location per day')
fig.update_yaxes(title_text='Pedestrian Count')
fig.update_layout(legend_title_text='Street name')
fig.show()

Additionally, we compute the average value per day.

In [30]:
mean_df = data_pre.groupby(['location_name', 'date'])['series'].mean().reset_index()
mean_df.head(5)

,location_name,date,series
0,Kaiserstraße,2024-01-01,312.608696
1,Kaiserstraße,2024-01-02,691.541667
2,Kaiserstraße,2024-01-03,801.041667
3,Kaiserstraße,2024-01-04,910.375000
4,Kaiserstraße,2024-01-05,1039.000000


In [31]:
fig = px.line(mean_df, x="date", y="series",color='location_name', title='Pedestrian count by location')
fig.update_yaxes(title_text='Pedestrian Count (mean)')
fig.update_layout(legend_title_text='Street name')
fig.show()

### Pedestrian Count vs. Temperature and Humidity

In [47]:
fig = make_subplots(rows=1, cols=2,
                    subplot_titles=('Pedestrian Count vs. Temperature', 'Pedestrian Count vs. Humidity'))

# Scatterplot Fußgängerzahlen gegen Temperatur
fig.add_trace(go.Scatter(x=data_pre['temp'], y=data_pre['series'], mode='markers', marker=dict(color='blue' , opacity=0.5), name='Temp'),
              row=1, col=1)

# Scatterplot Fußgängerzahlen gegen Luftfeuchtigkeit
fig.add_trace(go.Scatter(x=data_pre['humidity'], y=data_pre['series'], mode='markers', marker=dict(color='green', opacity=0.5), name='Humidity'),
              row=1, col=2)

# Layout anpassen
fig.update_xaxes(title_text='Temperature (°C)', row=1, col=1)
fig.update_xaxes(title_text='Humidity (%)', row=1, col=2)
fig.update_yaxes(title_text='Pedestrian Count', row=1, col=1)

fig.update_layout(title='Scatterplots: Pedestrian Count vs. Temperature and Humidity',
                  showlegend=True,
                  width=1000, height=500)

fig.show()

## Step 7: Generating Pandas Profiling Report

We will generate a Pandas Profiling report to obtain a comprehensive overview of the dataset.

The Pandas Profiling report provides us with detailed insights into the dataset, including:

- Overview: Summary statistics, descriptive statistics, and variable types.
- Variables: Distribution of each variable, correlation between variables, and missing values analysis.
- Interactions: Pairwise correlation, including scatter plots and correlation matrices.

Our dataset include 0% of missing values or duplicate rows, obtain 13897 of observation.

In [89]:
# Loading the dataset
data_pre_profiling = pd.read_csv("data\processed\eda_data.csv")

<>:2: SyntaxWarning:

invalid escape sequence '\p'

<>:2: SyntaxWarning:

invalid escape sequence '\p'

C:\Users\gavar\AppData\Local\Temp\ipykernel_6584\589351105.py:2: SyntaxWarning:

invalid escape sequence '\p'



In [91]:
profile = ProfileReport(data_pre_profiling)
profile.to_file("EDA.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]